In [1]:
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /content/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /content/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize


FileNotFoundError: ignored

In [3]:
!pip install imageio
!pip install torch torchvision

    100% |████████████████████████████████| 3.3MB 9.1MB/s 
    28% |█████████▏                      | 138.7MB 35.6MB/s eta 0:00:10

    100% |████████████████████████████████| 484.0MB 26kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5ca2a000 @  0x7f63dd5491c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 7.4MB/s 
    100% |████████████████████████████████| 2.0MB 15.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [4]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import os, tarfile
import imageio
from scipy.io import loadmat
# from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

import tensorflow as tf
print(tf.test.gpu_device_name())

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as datasets
import torchvision.transforms as transforms

/device:GPU:0


In [0]:
SVHN_URL_TRAIN = 'https://www.dropbox.com/s/k02n8imqlqx3wk1/train_32x32.mat?dl=1'
SVHN_URL_TEST = 'https://www.dropbox.com/s/et2dulb99ld6fez/test_32x32.mat?dl=1'

# Utility functions

#### ToDos
- Create a function to fetch data from a url.
- Check if it is already downloaded.
- Check if the file is csv or tar gz etc.
- Add cross-validation code to be able to use sklearn cross_val_score function to quickly evaluate the performance.

In [0]:
def fetch_data(URL, DOWNLOAD_FOLDER, DOWNLOAD_FILE):
  if not os.path.isdir(DOWNLOAD_FOLDER):
   os.makedirs(DOWNLOAD_FOLDER)
  
  if not os.path.isfile(DOWNLOAD_FOLDER+DOWNLOAD_FILE):
    print('Beginning file download...')
    urllib.request.urlretrieve(URL, DOWNLOAD_FOLDER+DOWNLOAD_FILE)
    print('Done.')
  
  svhn_data = loadmat(DOWNLOAD_FOLDER+DOWNLOAD_FILE)
  
  return svhn_data
  

In [0]:
def split_train_test(XY, n_splits=1, test_size=0.2, random_state=42):
    split = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    for train_index, test_index in split.split(XY[0], XY[1]):
        X_train, Y_train = XY[0][train_index,:], XY[1][train_index]
        X_test, Y_test = XY[0][test_index,:], XY[1][test_index]
        
    return X_train, Y_train, X_test, Y_test

In [0]:
def get_svhn_data(url, download_folder, download_file, split_data=False):
  
  
  svhn_dict = fetch_data(url, download_folder, download_file)
  
  
  X = svhn_dict['X']
  Y = svhn_dict['y']
  Y_new = np.zeros(Y.shape, dtype=np.float32)
  labels = np.arange(0,10,1)
  for i in labels[1:]:
    locs = np.where(Y==i)[0]
    Y_new[locs,:] = Y[locs,:]
    
  #locs = np.where(Y_train==10)[0]
  #Y_new[locs,:] = Y[locs,:]
  
  
  
  #X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)
  sz = X.shape
  X_new = np.zeros((sz[3], sz[2], sz[0], sz[1]), dtype=np.float32)
  for i in range(sz[3]):
    for j in range(sz[2]):
      X_new[i,j,:,:] = X[:,:,j,i]                   # <---- FOR PYTORCH (N x Channels x Width x Height)
    
  if split_data:
      return split_train_test((X_new, Y), n_splits=1, test_size=0.2, random_state=42)

  return X_new, Y_new

In [9]:
labels = np.arange(0,10,1)
print(labels[1:])

[1 2 3 4 5 6 7 8 9]


In [0]:
def one_hot_encoder(label):
    encoder = OneHotEncoder(dtype=np.float32)
    label_1hot = encoder.fit_transform(label.reshape(-1,1))
    print('The labels are: {}'.format(np.unique(label)))
    return label_1hot

# Load data

In [0]:
root_folder = 'drive/app/svhn/'
# root_folder = 'D:/dev/data/'

In [12]:
X_train, Y_train = get_svhn_data(SVHN_URL_TRAIN, root_folder, 'train_32x32.mat', 
                                 split_data=False)

X_test, Y_test = get_svhn_data(SVHN_URL_TEST, root_folder, 'test_32x32.mat', 
                               split_data=False)

# X_train, Y_train = get_svhn_data(svhn_dict=svhn_data, split_data=True)
print("Train: [{}, {}], Test: [{}, {}]".format(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape))
# print("Train: [{}, {}]".format(X_train.shape, Y_train.shape))

Beginning file download...
Done.
Beginning file download...
Done.
Train: [(73257, 3, 32, 32), (73257, 1)], Test: [(26032, 3, 32, 32), (26032, 1)]


In [13]:
!ls -l drive/app/svhn

total 240548
-rw-r--r-- 1 root root  64275384 May 15 17:32 test_32x32.mat
-rw-r--r-- 1 root root 182040794 May 15 17:32 train_32x32.mat


In [0]:
# #f, ax = plt.subplots(10,1,figsize=(150, 5))

# i=10
# #for i in range(10):
# idx = np.where(Y_train==i)[0][0]
# #print(idx)
# img = np.zeros([32,32,3])
# img[:,:,0] = X_train[idx,0,:,:]
# img[:,:,1] = X_train[idx,1,:,:]
# img[:,:,1] = X_train[idx,2,:,:]
# plt.imshow(img)
# plt.title(Y_train[idx,:])

In [15]:
Y_train_1hot = one_hot_encoder(Y_train).toarray().view(np.float32)
Y_test_1hot = one_hot_encoder(Y_test).toarray().view(np.float32)
# print(Y_train_1hot[0:2])
# print(type(Y_train_1hot))

The labels are: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
The labels are: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]


# IGNORE

In [0]:
def init_weights(in_features, out_features):
    W = np.random.randn(in_features, out_features) / sqrt(in_features)
    b = np.zeros(out_features)
    return W, b

In [0]:
class HiddenLinearLayer(object):
    def __init__(self, in_features, out_features, activation_fn):
        self.in_features = in_features
        self.out_features = out_features
        self.activation_fn = activation_fn
        
        W, b = init_weights(in_features, out_features)
        
        self.W = tf.Variable(W.astype(np.float32))
        self.b = tf.Variable(b.astype(np.float32))
        
        
    def forward(self, x):
        return self.activation_fn(tf.matmul(x, self.W) + self.b)
        

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

In [0]:
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [0]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [0]:
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

# Model definition

In [0]:
class CNN(nn.Module):
  
  def __init__(self, width, height, n_channels):
    super(CNN, self).__init__()
    
    self.conv_1 = nn.Conv2d(in_channels=n_channels, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.conv_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
    self.conv_3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
    self.conv_4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=1, padding=2)
    self.fc5 = nn.Linear(in_features=256*8*8, out_features=1024)
    self.fc6 = nn.Linear(in_features=1024, out_features=10)
    self.activation = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size=2)
    
    
  def forward(self, X):
    
    out = self.conv_1(X)
    out = self.activation(out)
    out = self.conv_2(out)
    out = self.activation(out)
    #print(out.size())
    out = self.maxpool(out)
    #print(out.size())
    out = self.conv_3(out)
    out = self.activation(out)
    out = self.conv_4(out)
    out = self.activation(out)
    #print(out.size())
    out = self.maxpool(out)
    #print(out.size())
    out = out.view(out.size(0), -1)
    out = self.fc5(out)
    out = self.activation(out)
    out = self.fc6(out)
    
    return out
  
  
  def fit(self, X, Y, criterion, optimizer, epochs, n_batches, batch_size, print_time):
    
    X = torch.from_numpy(X).double()
    Y = torch.from_numpy(Y).long()
    
    train_data = torch.utils.data.TensorDataset(X, Y)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    
    iteration = 0
    for epoch in range(epochs):
      for i, (x, y) in enumerate(train_loader):
        if torch.cuda.is_available():
          x = Variable(x.cuda())
          y = Variable(y.cuda())
        else:
          x = Variable(x)
          y = Variable(y)

        optimizer.zero_grad()
        outputs = self.forward(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        iteration += 1

        if iteration%print_time == 0:
            print('Epoch: {}, Iteration: {}, Loss: {}'.format(epoch, iteration, loss))
            #accuracy = predict(test_loader)
    
#     for epoch in range(epoches):
#       X_shuffled, Y_shuffled = shuffle(X, Y)
#       for ibatch in range(n_batches):
#         X_batch = torch.from_numpy(X_shuffled[ibatch*batch_size:(ibatch+1)*batch_size,:]).double()
#         Y_batch = torch.from_numpy(Y_shuffled[ibatch*batch_size:(ibatch+1)*batch_size,:]).double()
#         print(type(Y_batch))
        
#         if torch.cuda.is_available():
#           X_batch = Variable(X_batch.cuda()).float()
#           Y_batch = Variable(Y_batch.cuda()).type(torch.cuda.LongTensor)
#         else:
#           X_batch = Variable(X_batch).float()
#           Y_batch = Variable(Y_batch).type(torch.LongTensor)
          
#         optimizer.zero_grad()
#         outputs = self.forward(X_batch)
#         loss = cost_fn(outputs, Y_batch)
#         loss.backward()
#         optimizer.step()
        
#         if ibatch % print_time==0:
#           print('Epoch\Batch: {}\{}, Train loss: {}'.format(epoch, ibatch, loss))
          
          
  def predict(self, X, n_batches, batch_size):
    
    correct = 0
    test_cost = 0
    total = 0
    if ibatch%PRINT_TIME == 0:
      for ibatch_test in range(n_batches):
        X_batch = torch.from_numpy(X_test[ibatch_test*batch_size:(ibatch_test+1)*batch_size,:])
        
        outputs = self.forward(X_batch)
        if first == True:
          predicted = torch.argmax(outputs, dim=1)
          first = False
        else:
          predicted = torch.cat((predicted, torch.argmax(outputs, dim=1)))

    return predicted
  
  def score(self, Y, predicted):
    
    #predicted = torch.argmax(predicted, axis=1)
    accuracy = 100*np.mean(Y == predicted.data.numpy())
    return accuracy
        

In [0]:
WIDTH = 32
HEIGHT = 32
N_CHANNELS = 3
N_CLASSES = 10
BATCH_SIZE =32
MAX_ITER = 3
N_BATCHES = X_train.shape[0]//BATCH_SIZE
PRINT_TIME = N_BATCHES//N_BATCHES
TEST_N_BATCHES = X_test.shape[0]//BATCH_SIZE

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [0]:
ann = CNN(WIDTH, HEIGHT, N_CHANNELS)
ann = ann.double()
if torch.cuda.is_available():
  ann.cuda()
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(ann.parameters(), lr=0.001)

In [35]:
for parameter in list(ann.parameters()):
    print(parameter.size())

torch.Size([32, 3, 5, 5])
torch.Size([32])
torch.Size([64, 32, 5, 5])
torch.Size([64])
torch.Size([128, 64, 5, 5])
torch.Size([128])
torch.Size([256, 128, 5, 5])
torch.Size([256])
torch.Size([1024, 16384])
torch.Size([1024])
torch.Size([10, 1024])
torch.Size([10])


In [20]:
_, _, X_Select, Y_Select = split_train_test((X_train, Y_train), n_splits=1, test_size=0.4, random_state=42)
print(X_Select.shape, Y_Select.shape)

(29303, 3, 32, 32) (29303, 1)


In [21]:
ann.fit(X_train, np.squeeze(Y_train), cost_fn, optimizer, 10, N_BATCHES, BATCH_SIZE, PRINT_TIME)

RuntimeError: ignored

In [0]:
!kill -9 -1